In [1]:
import pandas as pd
import numpy as np
from sklearn.linear_model import LinearRegression, ElasticNet
from sklearn.preprocessing import PolynomialFeatures
from sklearn.neural_network import MLPRegressor
from sklearn.ensemble import RandomForestRegressor, GradientBoostingRegressor
from xgboost import XGBRegressor
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, LSTM, GRU, SimpleRNN, Conv1D, Flatten, Input
from tensorflow.keras.optimizers import Adam
from lightgbm import LGBMRegressor
from catboost import CatBoostRegressor
from sklearn.preprocessing import StandardScaler

In [3]:
# Cargar los datos desde los archivos proporcionados
costos_df = pd.read_excel('Costos_Subprocesos.xlsx')
indicadores_minas_df = pd.read_excel('Indicadores_Minas.xlsx')
indicadores_carguio_df = pd.read_excel('Indicadores_Carguio.xlsx')
indicadores_perforacion_df = pd.read_excel('Indicadores_Perforación.xlsx')
indicadores_transporte_df = pd.read_excel('Indicadores_Transporte.xlsx')

# Convertir las fechas a formato datetime en costos
costos_df['Subproceso_Costo'] = pd.to_datetime(costos_df['Subproceso_Costo'])

# Convertir las fechas a formato datetime en los DataFrames de indicadores
for df in [indicadores_minas_df, indicadores_carguio_df, indicadores_perforacion_df, indicadores_transporte_df]:
    df['Fecha'] = pd.to_datetime(df['Fecha'])

# Unir todos los DataFrames de indicadores en uno solo
indicadores_df = indicadores_minas_df.copy()
indicadores_df = pd.merge(indicadores_df, indicadores_carguio_df, on='Fecha', how='inner', suffixes=('_minas', '_carguio'))
indicadores_df = pd.merge(indicadores_df, indicadores_perforacion_df, on='Fecha', how='inner', suffixes=('', '_perforacion'))
indicadores_df = pd.merge(indicadores_df, indicadores_transporte_df, on='Fecha', how='inner', suffixes=('', '_transporte'))

# Unir el DataFrame de costos con el DataFrame combinado de indicadores
df_unificado = pd.merge(costos_df, indicadores_df, left_on='Subproceso_Costo', right_on='Fecha', how='inner')
df_unificado = df_unificado.drop(columns=['Fecha'])  # Eliminar la columna duplicada de Fecha

# Calcular la correlación entre todos los subprocesos y los indicadores usando Spearman
subprocesos_cols = costos_df.columns[1:]  # Excluyendo la columna de fecha
indicadores_cols = indicadores_df.columns[1:]  # Excluyendo la columna de fecha
correlation_matrix = df_unificado.corr(method='spearman')
correlation_filtered = correlation_matrix.loc[subprocesos_cols, indicadores_cols]
top_correlations = correlation_filtered.apply(lambda x: x.nlargest(3), axis=1)


# Crear una tabla con los subprocesos, indicadores más correlacionados y sus valores de correlación
resultados_lista = []
for subproceso in correlation_filtered.index:
    for indicador, correlacion in correlation_filtered.loc[subproceso].nlargest(3).items():
        resultados_lista.append({
            'Subproceso': subproceso,
            'Indicador': indicador,
            'Correlación': correlacion
        })

df_resultados = pd.DataFrame(resultados_lista)

# Guardar los resultados en un archivo Excel
df_resultados.to_excel('Mejor Indicador por Subproceso.xlsx', index=False)

# Continuar con la división de los datos para entrenamiento y prueba

# Dividir los datos en entrenamiento (2016-2021) y prueba (2022)
train_data = df_unificado[df_unificado['Subproceso_Costo'] < '2022-01-01']
test_data = df_unificado[(df_unificado['Subproceso_Costo'] >= '2022-01-01') & (df_unificado['Subproceso_Costo'] < '2023-01-01')]

In [5]:
# Definir una lista de modelos a evaluar
modelos = {
    'Linear Regression': LinearRegression(),
    'Polynomial Regression': LinearRegression(),  # Usaremos PolynomialFeatures más adelante
    'ElasticNet': ElasticNet(alpha=1.0, l1_ratio=0.5, random_state=42)
}

# Crear un diccionario para almacenar los resultados de cada modelo
resultados_modelos = {}

# Realizar predicciones para cada modelo
for nombre_modelo, modelo in modelos.items():
    resultados = {}
    for subproceso in subprocesos_cols:
        indicadores_seleccionados = top_correlations.loc[subproceso].dropna().index.tolist()
        if len(indicadores_seleccionados) > 0:
            X_train = train_data[indicadores_seleccionados]
            y_train = train_data[subproceso]
            X_test = test_data[indicadores_seleccionados]
            
            if nombre_modelo == 'Polynomial Regression':
                poly = PolynomialFeatures(degree=2)
                X_train = poly.fit_transform(X_train)
                X_test = poly.transform(X_test)
                modelo.fit(X_train, y_train)
            else:
                modelo.fit(X_train, y_train)
            
            y_pred_2022 = modelo.predict(X_test)
            costo_real_2022 = test_data[subproceso].values
            
            # Calcular las diferencias y diferencias porcentuales
            diferencias = y_pred_2022 - costo_real_2022
            diferencias_pct = [(dif / real) * 100 if real != 0 else 'N/A' for dif, real in zip(diferencias, costo_real_2022)]
            
            # Guardar resultados en el diccionario
            for mes, pred, real, dif, dif_pct in zip(pd.date_range('2022-01-01', '2022-12-01', freq='MS').strftime("%B-%Y"), y_pred_2022, costo_real_2022, diferencias, diferencias_pct):
                resultados.setdefault(subproceso, {}).update({
                    f'Predicción {mes}': pred,
                    f'Costo Real {mes}': real,
                    f'Diferencia {mes}': dif,
                    f'Diferencia % {mes}': dif_pct
                })
    
    # Convertir los resultados a un DataFrame
    resultados_df = pd.DataFrame(resultados).T
    resultados_modelos[nombre_modelo] = resultados_df

# Los DataFrames de comparación de modelos y exportación se mantendrán iguales
mejores_modelos_df = pd.DataFrame(index=subprocesos_cols, columns=pd.date_range('2022-01-01', '2022-12-01', freq='MS').strftime("%B-%Y"))
diferencias_df = pd.DataFrame(index=subprocesos_cols, columns=pd.date_range('2022-01-01', '2022-12-01', freq='MS').strftime("%B-%Y"))

# Comparar los resultados entre modelos para cada subproceso y mes
for subproceso in subprocesos_cols:
    for mes in pd.date_range('2022-01-01', '2022-12-01', freq='MS').strftime("%B-%Y"):
        mejor_modelo = None
        menor_diferencia = np.inf
        
        for nombre_modelo, df in resultados_modelos.items():
            diferencia_col = f'Diferencia % {mes}'
            if diferencia_col in df.columns and subproceso in df.index and df.loc[subproceso, diferencia_col] != 'N/A':
                diferencia = abs(df.loc[subproceso, diferencia_col])
                if diferencia < menor_diferencia:
                    menor_diferencia = diferencia
                    mejor_modelo = nombre_modelo
        
        # Guardar el mejor modelo y su diferencia
        mejores_modelos_df.loc[subproceso, mes] = mejor_modelo
        diferencias_df.loc[subproceso, mes] = menor_diferencia

# Exportar los resultados a un archivo Excel con dos hojas
output_file_path = 'Resultados_Etapa1.xlsx'

with pd.ExcelWriter(output_file_path) as writer:
    mejores_modelos_df.to_excel(writer, sheet_name='Mejores Modelos')
    diferencias_df.to_excel(writer, sheet_name='Diferencias %')
    resultados_df.to_excel(writer, sheet_name='Global')

NameError: name 'top_correlations' is not defined

In [25]:
# Definir la lista de modelos a evaluar
modelos = {
    'Random Forest': RandomForestRegressor(n_estimators=100, random_state=42),
    'Gradient Boosting': GradientBoostingRegressor(n_estimators=100, random_state=42),
    'XGBoost': XGBRegressor(n_estimators=100, random_state=42),
    'LightGBM': LGBMRegressor(n_estimators=100, random_state=42),
    'CatBoost': CatBoostRegressor(n_estimators=100, random_state=42, verbose=0)
}

# Crear un diccionario para almacenar los resultados de cada modelo
resultados_modelos = {}

# Modificación del código para evitar el entrenamiento en subprocesos sin variabilidad

# Crear un diccionario para almacenar los resultados de cada modelo
resultados_modelos = {}

# Realizar predicciones para cada modelo
for nombre_modelo, modelo in modelos.items():
    resultados = {}
    for subproceso in subprocesos_cols:
        indicadores_seleccionados = top_correlations.loc[subproceso].dropna().index.tolist()
        if len(indicadores_seleccionados) > 0:
            X_train = train_data[indicadores_seleccionados]
            y_train = train_data[subproceso]
            X_test = test_data[indicadores_seleccionados]
            
            # Verificar si todos los valores de y_train son iguales
            if y_train.nunique() > 1:
                modelo.fit(X_train, y_train)
                y_pred_2022 = modelo.predict(X_test)
                costo_real_2022 = test_data[subproceso].values
                
                # Calcular las diferencias y diferencias porcentuales
                diferencias = y_pred_2022 - costo_real_2022
                diferencias_pct = [(dif / real) * 100 if real != 0 else 'N/A' for dif, real in zip(diferencias, costo_real_2022)]
                
                # Guardar resultados en el diccionario
                for mes, pred, real, dif, dif_pct in zip(pd.date_range('2022-01-01', '2022-12-01', freq='MS').strftime("%B-%Y"), y_pred_2022, costo_real_2022, diferencias, diferencias_pct):
                    resultados.setdefault(subproceso, {}).update({
                        f'Predicción {mes}': pred,
                        f'Costo Real {mes}': real,
                        f'Diferencia {mes}': dif,
                        f'Diferencia % {mes}': dif_pct
                    })
            else:
                # Asignar un valor predeterminado si no hay variabilidad
                for mes in pd.date_range('2022-01-01', '2022-12-01', freq='MS').strftime("%B-%Y"):
                    resultados.setdefault(subproceso, {}).update({
                        f'Predicción {mes}': y_train.iloc[0],  # Valor constante
                        f'Costo Real {mes}': 'No Variability',
                        f'Diferencia {mes}': 0,
                        f'Diferencia % {mes}': 0
                    })
    
    # Convertir los resultados a un DataFrame
    resultados_df = pd.DataFrame(resultados).T
    resultados_modelos[nombre_modelo] = resultados_df

# Los DataFrames de comparación de modelos y exportación se mantendrán iguales
mejores_modelos_df = pd.DataFrame(index=subprocesos_cols, columns=pd.date_range('2022-01-01', '2022-12-01', freq='MS').strftime("%B-%Y"))
diferencias_df = pd.DataFrame(index=subprocesos_cols, columns=pd.date_range('2022-01-01', '2022-12-01', freq='MS').strftime("%B-%Y"))

# Comparar los resultados entre modelos para cada subproceso y mes
for subproceso in subprocesos_cols:
    for mes in pd.date_range('2022-01-01', '2022-12-01', freq='MS').strftime("%B-%Y"):
        mejor_modelo = None
        menor_diferencia = np.inf
        
        for nombre_modelo, df in resultados_modelos.items():
            diferencia_col = f'Diferencia % {mes}'
            if diferencia_col in df.columns and subproceso in df.index and df.loc[subproceso, diferencia_col] != 'N/A':
                diferencia = abs(df.loc[subproceso, diferencia_col])
                if diferencia < menor_diferencia:
                    menor_diferencia = diferencia
                    mejor_modelo = nombre_modelo
        
        # Guardar el mejor modelo y su diferencia
        mejores_modelos_df.loc[subproceso, mes] = mejor_modelo
        diferencias_df.loc[subproceso, mes] = menor_diferencia

# Exportar los resultados a un archivo Excel con dos hojas
output_file_path = 'Resultados_Etapa2.xlsx'

with pd.ExcelWriter(output_file_path) as writer:
    mejores_modelos_df.to_excel(writer, sheet_name='Mejores Modelos')
    diferencias_df.to_excel(writer, sheet_name='Diferencias %')
    resultados_df.to_excel(writer, sheet_name='Global')

output_file_path

[LightGBM] [Warning] Found whitespace in feature_names, replace with underlines
[LightGBM] [Info] Auto-choosing col-wise multi-threading, the overhead of testing was 0.000021 seconds.
You can set `force_col_wise=true` to remove the overhead.
[LightGBM] [Info] Total Bins 75
[LightGBM] [Info] Number of data points in the train set: 71, number of used features: 3
[LightGBM] [Info] Start training from score 2096037.888204
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive 

'Resultados_Etapa2.xlsx'

In [18]:
# Preprocesar los datos para redes neuronales
scaler = StandardScaler()
X_train_scaled = scaler.fit_transform(train_data[indicadores_cols])
X_test_scaled = scaler.transform(test_data[indicadores_cols])

# Reshape data for RNN, LSTM, GRU, and CNN models (requires 3D input)
X_train_rnn = X_train_scaled.reshape((X_train_scaled.shape[0], X_train_scaled.shape[1], 1))
X_test_rnn = X_test_scaled.reshape((X_test_scaled.shape[0], X_test_scaled.shape[1], 1))

# Definir modelos en un diccionario para iterar sobre ellos
def create_mlp_model(input_shape):
    model = Sequential()
    model.add(Dense(64, input_shape=(input_shape,), activation='relu'))
    model.add(Dense(32, activation='relu'))
    model.add(Dense(1))
    model.compile(optimizer='adam', loss='mse')
    return model

def create_rnn_model(input_shape):
    model = Sequential()
    model.add(SimpleRNN(50, input_shape=(input_shape[0], input_shape[1]), activation='relu'))
    model.add(Dense(1))
    model.compile(optimizer='adam', loss='mse')
    return model

def create_lstm_model(input_shape):
    model = Sequential()
    model.add(LSTM(50, input_shape=(input_shape[0], input_shape[1]), activation='relu'))
    model.add(Dense(1))
    model.compile(optimizer='adam', loss='mse')
    return model

def create_gru_model(input_shape):
    model = Sequential()
    model.add(GRU(50, input_shape=(input_shape[0], input_shape[1]), activation='relu'))
    model.add(Dense(1))
    model.compile(optimizer='adam', loss='mse')
    return model

def create_cnn_model(input_shape):
    model = Sequential()
    model.add(Conv1D(64, kernel_size=2, activation='relu', input_shape=(input_shape[0], input_shape[1])))
    model.add(Flatten())
    model.add(Dense(1))
    model.compile(optimizer='adam', loss='mse')
    return model

# Definir los modelos y sus configuraciones
modelos = {
    'MLP': create_mlp_model(input_shape=X_train_scaled.shape[1]),
    'RNN': create_rnn_model(input_shape=(X_train_rnn.shape[1], X_train_rnn.shape[2])),
    'LSTM': create_lstm_model(input_shape=(X_train_rnn.shape[1], X_train_rnn.shape[2])),
    'GRU': create_gru_model(input_shape=(X_train_rnn.shape[1], X_train_rnn.shape[2])),
    'CNN': create_cnn_model(input_shape=(X_train_rnn.shape[1], X_train_rnn.shape[2]))
}

# Configuración del número de épocas para el entrenamiento
epochs = 50

# Entrenar los modelos y almacenar los resultados
resultados_modelos = {}

for nombre_modelo, model in modelos.items():
    resultados = {}
    for subproceso in subprocesos_cols:
        y_train = train_data[subproceso].values
        y_test = test_data[subproceso].values
        
        if nombre_modelo == 'MLP':
            model.fit(X_train_scaled, y_train, epochs=epochs, verbose=0)
            y_pred_2022 = model.predict(X_test_scaled).flatten()
        else:
            model.fit(X_train_rnn, y_train, epochs=epochs, verbose=0)
            y_pred_2022 = model.predict(X_test_rnn).flatten()

        # Calcular las diferencias y diferencias porcentuales
        diferencias = y_pred_2022 - y_test
        diferencias_pct = [(dif / real) * 100 if real != 0 else 'N/A' for dif, real in zip(diferencias, y_test)]
        
        # Guardar resultados en el diccionario
        for mes, pred, real, dif, dif_pct in zip(pd.date_range('2022-01-01', '2022-12-01', freq='MS').strftime("%B-%Y"), y_pred_2022, y_test, diferencias, diferencias_pct):
            resultados.setdefault(subproceso, {}).update({
                f'Predicción {mes}': pred,
                f'Costo Real {mes}': real,
                f'Diferencia {mes}': dif,
                f'Diferencia % {mes}': dif_pct
            })
    
    # Convertir los resultados a un DataFrame
    resultados_df = pd.DataFrame(resultados).T
    resultados_modelos[nombre_modelo] = resultados_df

# Los DataFrames de comparación de modelos y exportación se mantendrán iguales
mejores_modelos_df = pd.DataFrame(index=subprocesos_cols, columns=pd.date_range('2022-01-01', '2022-12-01', freq='MS').strftime("%B-%Y"))
diferencias_df = pd.DataFrame(index=subprocesos_cols, columns=pd.date_range('2022-01-01', '2022-12-01', freq='MS').strftime("%B-%Y"))

# Comparar los resultados entre modelos para cada subproceso y mes
for subproceso in subprocesos_cols:
    for mes in pd.date_range('2022-01-01', '2022-12-01', freq='MS').strftime("%B-%Y"):
        mejor_modelo = None
        menor_diferencia = np.inf
        
        for nombre_modelo, df in resultados_modelos.items():
            diferencia_col = f'Diferencia % {mes}'
            if diferencia_col in df.columns and subproceso in df.index and df.loc[subproceso, diferencia_col] != 'N/A':
                diferencia = abs(df.loc[subproceso, diferencia_col])
                if diferencia < menor_diferencia:
                    menor_diferencia = diferencia
                    mejor_modelo = nombre_modelo
        
        # Guardar el mejor modelo y su diferencia
        mejores_modelos_df.loc[subproceso, mes] = mejor_modelo
        diferencias_df.loc[subproceso, mes] = menor_diferencia

# Exportar los resultados a un archivo Excel con dos hojas
output_file_path = 'Resultados_Etapa3.xlsx'

with pd.ExcelWriter(output_file_path) as writer:
    mejores_modelos_df.to_excel(writer, sheet_name='Mejores Modelos')
    diferencias_df.to_excel(writer, sheet_name='Diferencias %')

output_file_path

C:\Users\Administrator\AppData\Local\Programs\Python\Python312\Lib\site-packages\keras\src\layers\core\dense.py:87: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(activity_regularizer=activity_regularizer, **kwargs)
C:\Users\Administrator\AppData\Local\Programs\Python\Python312\Lib\site-packages\keras\src\layers\rnn\rnn.py:204: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(**kwargs)
C:\Users\Administrator\AppData\Local\Programs\Python\Python312\Lib\site-packages\keras\src\layers\convolutional\base_conv.py:107: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the mo

1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 33ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 14ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 14ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 14ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 14ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 15ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 13ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 15ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 14ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 14ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 15ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 14ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 14ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 13ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 13ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 14ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 14ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 15ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 15ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 14ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 13ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 15ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 14ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 13ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 14ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 14ms/step
1/1 ━━━━━━━━

'Resultados_Etapa3.xlsx'